# Preliminary test on a Single Bank

In [251]:
import os
import pdftables_api
import glob2
import tabula
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import cv2
import matplotlib.pyplot as plt
import re
import logging

In [2]:
pd.set_option("max_colwidth", 3000)
pd.set_option("max_rows", 2000)
pd.set_option("max_columns", 2000)

# 1. Statement extraction

In [3]:
DATA_FOLDER = './statements/'
MY_API_KEY = ''

In [4]:
glob2.glob(os.path.join(DATA_FOLDER, "*"))

['./statements/224912180637606770417463100932.pdf',
 './statements/Account Statement (1).pdf',
 './statements/420 STATEMENT.pdf',
 './statements/CUSTOMER_STATEMENT_OF_ACCT_ONLINE_F12[NWOLU].pdf',
 './statements/0163041071-(205154861110).pdf',
 './statements/account_stmt_24Apr2021_24Jun2021.pdf',
 './statements/68197656963758929248882067034.pdf',
 './statements/OFFER LETTER MR FITZGERALD Signed 01072021.pdf',
 './statements/output.csv',
 './statements/420 PROPERTIES LIMITED.pdf',
 './statements/fav Ajo.pdf',
 './statements/statement_adeshina iliasu.pdf',
 './statements/224912180637606770417463100932output.csv',
 './statements/68197656963758929248882067034output.csv']

In [5]:
def process_bank_statements(b_statements_gt_bank, out_format ='csv'):
    '''
    Method to transform to csv a list of Bank statements paths
    '''
    
    n_statements = len(b_statements_gt_bank)
    for bk_st in tqdm(b_statements_gt_bank):
        inp = bk_st
        out = bk_st.replace(".pdf","output.csv")
        tabula.convert_into(inp, out, output_format=out_format)
        
    print( f"{n_statements} bank_statements processed!")

In [6]:
bank_statements_gt_bank = ["./statements/224912180637606770417463100932.pdf",
                          "./statements/68197656963758929248882067034.pdf"]

In [7]:
process_bank_statements(bank_statements_gt_bank, out_format ='csv')

100%|██████████| 2/2 [00:03<00:00,  1.56s/it]

2 bank_statements processed!


In [8]:
def bank_statement_to_dataframe(b_statement_gt_bank, mult=True, pages_all='all'):
    '''
    Function to transform to dataframe a Bank statement pdf
    
    b_statement_gt_bank: a statement path of a Bank statement pdf
    mult: consider various tables in the document
    pages_all: keep all pages if 'all'
    '''
    df = tabula.read_pdf(b_statement_gt_bank,
                         multiple_tables=mult,
                         pages=pages_all)
    return df

In [9]:
help(tabula.read_pdf)

Help on function read_pdf in module tabula.io:

read_pdf(input_path, output_format=None, encoding='utf-8', java_options=None, pandas_options=None, multiple_tables=True, user_agent=None, **kwargs)
    Read tables in PDF.
    
    Args:
        input_path (str, path object or file-like object):
            File like object of tareget PDF file.
            It can be URL, which is downloaded by tabula-py automatically.
        output_format (str, optional):
            Output format for returned object (``dataframe`` or ``json``)
        encoding (str, optional):
            Encoding type for pandas. Default: ``utf-8``
        java_options (list, optional):
            Set java options.
    
            Example:
                ``["-Xmx256m"]``
        pandas_options (dict, optional):
            Set pandas options.
    
            Example:
                ``{'header': None}``
    
            Note:
                With ``multiple_tables=True`` (default), pandas_options is passed
        

In [10]:
def bank_statement_to_dataframe_plus(b_statement_gt_bank, mult=True, pages_all='all'):
    '''
    Function to transform to dataframe a Bank statement pdf
    
    b_statement_gt_bank: a statement path of a Bank statement pdf
    mult: consider various tables in the document
    pages_all: keep all pages if 'all'
    '''
    df = tabula.read_pdf(b_statement_gt_bank,
                         multiple_tables=mult,
                         lattice= True,
                         pages=pages_all)
    return df

for the other type:
read as normal to extract the 2 info tables
and then read as lattice to extract the remaining info

In [11]:
df = tabula.read_pdf("./GT_BANK/orange_header/Abiodun_lanre_gtb_statement.pdf",
                     multiple_tables=True,
                     lattice = True,
                     pages='all')

In [ ]:
df[50]

In [ ]:
df = tabula.read_pdf("./statements/224912180637606770417463100932.pdf", 
                     multiple_tables=True,
                    pages='all')

In [ ]:
b_statement = bank_statements_gt_bank[1]
customer = bank_statement_to_dataframe(b_statement, mult=True, pages_all='all')

In [ ]:
type_account = type_account.T.reset_index().T.reset_index(drop=True)
type_account.columns = ["PI", "Detail"]

In [ ]:
account_customer_description = type_account[type_account["PI"]== "Account Type"]["Detail"].values[0]

### Refactor idea: 
- use abstract classes
- use decorators

### Abstract classes:

In [ ]:
# On this example, we define a class Bank_statement
# this class is an abstract class. That is, it describes
# the list of methods / functions that the class implements

In [ ]:
from abc import ABC, abstractmethod
 
class Bank_statement(ABC):
 
    def __init__(self, b_statement_bank, bank_id):
        self.b_statement_bank = b_statement_bank
        self.bank_id = bank_id
        super().__init__()
    
    @abstractmethod
    def return_bank(self):
        pass
    
    @abstractmethod
    # when you write an abstract class you are not
    # forced to declare the arguments in the abstract function
    def process_bank(self):
        pass

In [ ]:
class _GT_BankStatement(Bank_statement):
    def __init__(self, b_gt_statement_bank, bank_id):
        self.b_statement_bank = b_gt_statement_bank
        self.bank_id = bank_id
        #super().__init__()
    
    def return_bank(self):
        return self.bank_id
    
    def process_bank(self, bk_statement):
        return "b_gt_bank"

In [ ]:
# The class can be instantiated
# The class's methods can also be called

In [ ]:
gt = _GT_BankStatement("my_file.csv", 15)

In [ ]:
gt.process_bank(2)

### Decorators

In [ ]:
def time_decorator(decorated_func):
    # the exterior  part returns the interior part
    # the interior part  runs the function we want to decorate
    # plus its decorations
    
    def timely_decorate_me():
        
        # this decoration happens here
        # the logic is to enclose the code with 2 timer ticks
        # start tick
        start_timer = time.time()
        
        # you need to evaluate the function
        interm = decorated_func()
        
        # end tick
        end_timer = time.time()
        
        # the goal is to display how long the function took
        print(end_timer-start_timer)
        
        return interm
    
    return timely_decorate_me

In [ ]:
@time_decorator
def my_function():
    '''
    simple function which stores in a list
    1000 times the same piece of text
    '''
    l = []
    for i in range(1000):
        l.append("this is crazy")
    return "done"

In [ ]:
my_function()

In [ ]:
os.getcwd()

In [ ]:
import pdf2image
def safe_convert(filename, out, folder):
    assert os.path.exists(filename)
    my_im = pdf2image.convert_from_path(filename,output_folder = folder, last_page= 1, output_file= out, paths_only= True, fmt='jpg')
    return my_im

In [ ]:
# step 0:
# conversion to images

In [ ]:
def convert_pdf_2_im(folder,filename):
    full_filename = os.path.join(folder,filename)
    print(full_filename)
    out = full_filename.split(".pdf")[0].split("/")[-1]
    safe_convert(full_filename, out, folder)

In [ ]:
folder = "./GT_BANK/no_header"
filenames = [os.path.basename(el) for el in glob2.glob(os.path.join(folder,"*.pdf"))]

In [ ]:
filenames

In [ ]:
for f in filenames:
    convert_pdf_2_im(folder,f)

In [ ]:
def read_image_crop(folder, image_path, crop=250):
    '''
    the function crops the right upper corner
    '''
    my_im = cv2.imread(os.path.join(folder, image_path))
    my_im_RGB = cv2.cvtColor(my_im, cv2.COLOR_BGR2RGB)
    return my_im_RGB[:250,-250:]

In [84]:
my_folder = "./GT_BANK/no_header"
my_image_path =  [os.path.basename(el) for el in glob2.glob(os.path.join(folder,"*.jpg"))]

NameError: name 'folder' is not defined

In [ ]:
for im in my_image_path:
    plt.imshow(read_image_crop(my_folder, im))
    plt.show()

## Second type of gt bank document

In [93]:
class GT_BankStatement_header():

    def __init__(self, b_statement_gt_bank, bank_id='GT_Bank', template_version=1):
        '''
        initializes the Gt_bank statement class
        '''
        # a bankstatement has 2 input attributes (for now)
        # bank_id & statement pdf path
        
        self.bank_id = bank_id
        self.customer_bank_statement_formatted = bank_statement_to_dataframe_plus(b_statement_gt_bank, mult=True, pages_all='all')
        self.customer_bank_statement_formatted_statement = [ df for df in self.customer_bank_statement_formatted if len(df.columns)>2 ]
        #self.customer_bank_statement_raw = bank_statement_to_dataframe(b_statement_gt_bank, mult=True, pages_all='all')
      
        self.df = self.customer_bank_statement_formatted.copy()
        #self.raw_df = self.customer_bank_statement_raw.copy()

    def describe_accounts_types_header(self):
        '''
        returns the account type (savings or current)
        '''
        
        info_idxs = []
        for i in range(len(self.df)):
            if len(self.df[i].columns) <=2:
                 info_idxs.append(i)
        self.accounts = []
        print(info_idxs)
        
        for idx,val in enumerate(info_idxs):

            df = self.df[idx].transpose().reset_index()
            try:
                df.columns = [0,1,2]
                arr_df =[]
                for i in range(3):
                    arr_df.append(df[i])
                for df_str in arr_df:
                    #print(df_str.values[0])
                    
                    if "\rAccount" in df_str.values[0]:
                        self.accounts.append(df_str.values[0])      
                        
            except Exception as e:
                pass
                #print(e)
                #print(self.df[idx])
        
        self.types_account = set((['current' for el in customer2.accounts if 'CURRENT' in el ])+(['savings' for el in customer2.accounts if 'SAVINGS' in el ]))
  
    def combine_df(self):
        
        all_statements = []
        for statement in self.customer_bank_statement_formatted_statement:
            all_statements.append(pd.DataFrame(statement).T.reset_index().T.replace(regex={r'Unnamed:':''}))
        self.combined = pd.concat(all_statements).reset_index(drop=True)
        sef.combined.columns = ['Trans. Date','Reference','Value. Date','Debits','Credits','Balance','Remarks_processed']


In [86]:
customer2 =  GT_BankStatement_header("./GT_BANK/orange_header/0163041071-(205154861110).pdf")

In [87]:
customer2.describe_accounts_types_header()

[0, 1, 28]


In [88]:
customer2.types_account

{'current'}

In [89]:
customer2.combine_df()

In [90]:
customer2.combined

,0,1,2,3,4,5,6
0,01­Dec­2020,0,01­Dec­2020.1,"20,000.00",0,"3,993.22",OWN ACCOUNT TRANSFER\rREF:205154861000000200002...
1,01­Dec­2020,0,01­Dec­2020,"2,000.00",NaN,"1,993.22","TRANSFER BETWEEN\rCUSTOMERS Via USSD GTBank\rTransfer\r0000000020501548612320696...\rfrom ADEOSUN KEHINDE WAKIL\rto ODELE,ABIODUN RAYMOND"
2,01­Dec­2020,0,01­Dec­2020,NaN,"58,546.49","60,539.71",MTHLY SALARY 13th Month 2020
3,01­Dec­2020,0,01­Dec­2020,"41,000.00",NaN,"19,539.71","TRANSFER BETWEEN\rCUSTOMERS Via USSD GTBank\rTransfer\r0000000020501548613710356...\rfrom ADEOSUN KEHINDE WAKIL\rto ADEOSUN, ABDULSEMIU G."
4,01­Dec­2020,0,01­Dec­2020,"4,000.00",NaN,"15,539.71",NIBSS Instant Payment Outward\r0000132012011932440002717...\rUSSD NIP Transfer from\r08033725563 TO\rZBN/TEMITAYO ALIU AIBINU\rREF:000013201201193244000...
5,01­Dec­2020,0,01­Dec­2020,200.00,NaN,"15,339.71",Airtime Purchase USSD­\r101CT0000000002185428097­\r2348033725563
6,01­Dec­2020,0,01­Dec­2020,"1,000.00",NaN,"14,339.71",Airtime Purchase USSD­\r101CT0000000002185430572­\r2348031164646
7,01­Dec­2020,0,01­Dec­2020,"1,000.00",NaN,"13,339.71",Airtime Purchase USSD­\r101CT0000000002185792084­\r2348180043134
8,01­Dec­2020,0,01­Dec­2020,NaN,"3,000.00","16,339.71",OWN ACCOUNT TRANSFER\rREF:205154861000000030002...
9,01­Dec­2020,4,01­Dec­2020,"10,000.00",NaN,"6,339.71",STANDING ORDER Standing\rOrder via Internet Banking food\rstuff


In [92]:
customer2.combined

,0,1,2,3,4,5,6
0,01­Dec­2020,0,01­Dec­2020.1,"20,000.00",0,"3,993.22",OWN ACCOUNT TRANSFER\rREF:205154861000000200002...
1,01­Dec­2020,0,01­Dec­2020,"2,000.00",NaN,"1,993.22","TRANSFER BETWEEN\rCUSTOMERS Via USSD GTBank\rTransfer\r0000000020501548612320696...\rfrom ADEOSUN KEHINDE WAKIL\rto ODELE,ABIODUN RAYMOND"
2,01­Dec­2020,0,01­Dec­2020,NaN,"58,546.49","60,539.71",MTHLY SALARY 13th Month 2020
3,01­Dec­2020,0,01­Dec­2020,"41,000.00",NaN,"19,539.71","TRANSFER BETWEEN\rCUSTOMERS Via USSD GTBank\rTransfer\r0000000020501548613710356...\rfrom ADEOSUN KEHINDE WAKIL\rto ADEOSUN, ABDULSEMIU G."
4,01­Dec­2020,0,01­Dec­2020,"4,000.00",NaN,"15,539.71",NIBSS Instant Payment Outward\r0000132012011932440002717...\rUSSD NIP Transfer from\r08033725563 TO\rZBN/TEMITAYO ALIU AIBINU\rREF:000013201201193244000...
5,01­Dec­2020,0,01­Dec­2020,200.00,NaN,"15,339.71",Airtime Purchase USSD­\r101CT0000000002185428097­\r2348033725563
6,01­Dec­2020,0,01­Dec­2020,"1,000.00",NaN,"14,339.71",Airtime Purchase USSD­\r101CT0000000002185430572­\r2348031164646
7,01­Dec­2020,0,01­Dec­2020,"1,000.00",NaN,"13,339.71",Airtime Purchase USSD­\r101CT0000000002185792084­\r2348180043134
8,01­Dec­2020,0,01­Dec­2020,NaN,"3,000.00","16,339.71",OWN ACCOUNT TRANSFER\rREF:205154861000000030002...
9,01­Dec­2020,4,01­Dec­2020,"10,000.00",NaN,"6,339.71",STANDING ORDER Standing\rOrder via Internet Banking food\rstuff


In [522]:
def log_info(step, account_type, bank_id, statement_gt_bank, error,**kwargs):
    
    extension = os.path.splitext(f'{statement_gt_bank}')[1][:]
    rel_logname = f'{statement_gt_bank}'.replace(extension,".log").split("/")[-1]
    logname = os.path.join(os.getcwd(), rel_logname)
    
    FORMAT = '%(asctime)-15s %(bank_account_type)s %(bank_file_name)-15s %(error_message)s %(message)s'
    #FORMAT = '%(asctime)-15s %(message)s'
    logging.basicConfig(filename=logname,
                        filemode='a',
                        format=FORMAT,
                        force=True,
                        level=logging.INFO)

    d = {
        'bank_account_type': account_type,
        'step':step,
        'bank_id': bank_id, 
        'bank_file_name': statement_gt_bank,
        'error_message': str(error)
        }

    
    logger = logging.getLogger('bank_statement')
    if error != None:
        logger.info('ETL_STEP_%s', step, extra=d)
    else:
        logger.error('ETL_STEP_%s', step, extra=d)

In [523]:
class BankAccountTypeError(Exception):
    
    def __init__(self, bank_account_type, message="There are too many uploaded documents"):
        self.bank_account_type_list = bank_account_type
        self.message = message
        super().__init__(self.message)
        
    def __str__(self):
        return "You must upload one statement per individual (SAVINGS/CURRENT) bank statement per individual"

In [524]:
class BankStatementPathError(Exception):
    
    def __init__(self, message="The Bank statement path does not exist"):
        self.message = message
        super().__init__(self.message)
        
    def __str__(self):
        return "Check that there is no typo in the bank statement filepath"

In [525]:
class BankStatementFormatError(Exception):
    
    def __init__(self, message="The Bank statement format is not accepted"):
        self.message = message
        super().__init__(self.message)
        
    def __str__(self):
        return "The valid format is pdf - only"

In [526]:
class BankStatementColumnsError(Exception):
    
    def __init__(self, template_version, bank_id, mapping):

        
        message = f"DataFrames format should have at max: {mapping[bank_id][template_version]}"
        self.message = message
        super().__init__(self.message)
        
    def __str__(self):
        return "There is a problem of columns during extraction"

In [619]:
class GT_BankStatement_no_header():
    
    def __init__(self, b_statement_gt_bank, bank_id='GT_Bank', template_version='NO_HEADER'):
        '''
        initializes the Gt_bank statement class
        '''
        # a bankstatement has 2 input attributes (for now)
        # bank_id & statement pdf path
        
        self.bank_id = bank_id
        self.filename = b_statement_gt_bank
        self.allowed_extensions =["pdf"]
        self.template_version = template_version
        
                
        self.mapping = {
            "GT_Bank": {
                "NO_HEADER":8,
                "HEADER":7
                }
        }
        
        self.log_check_bank_statement_extension()
        self.log_check_bank_statement_path()
        
        self.customer_bank_statement = bank_statement_to_dataframe(b_statement_gt_bank, mult=True, pages_all='all')
        self.df = self.customer_bank_statement.copy()
        
              
    def describe_accounts_types(self):
        '''
        returns the account type (savings or current)
        '''
        
        self.accounts_info = [df for df in self.df if len(df.columns) == 2]
        account_type_list= []
        for account in self.accounts_info:
            account.index = account["Print. Date"]
            acc_tr = account.transpose()
            account_type_list.append(acc_tr["Account Type"][-1])
        return account_type_list
            
        
        
    def log_check_bank_statement_extension(self):
        extension = os.path.splitext(self.filename)[1][1:]
        if extension not in self.allowed_extensions:
            account_type = 'NA'
            log_info('PRELOAD',account_type, self.bank_id, self.filename,'BankStatementFormatError')
            raise BankStatementFormatError
            
            
    def log_check_bank_statement_path(self):
        file_exists = os.path.exists(self.filename)
        if not file_exists:
            account_type = 'NA'
            log_info('PRELOAD',account_type, self.bank_id, self.filename,'BankStatementPathError')
            raise BankStatementPathError
        
        
    def log_check_account_type(self):
        '''
        checks the bank_type and logs the operation
        '''
        
        account_type = self.describe_accounts_types()
        if len(account_type) != 1:
            log_info('LOAD',account_type, self.bank_id, self.filename,'BankAccountTypeError')
            raise BankAccountTypeError(account_type)
        else:
            self.account_type = account_type[0]
            log_info('LOAD',self.account_type, self.bank_id, self.filename,None)
            
            
    def log_check_table(self):
        '''
        Store the list of columns names of all extracted dataframes
        '''
        # store the columns names for debug
        self.df_cols = [b_statement_dataframe.columns for b_statement_dataframe in self.df]
        
        max_ncols = self.mapping[self.bank_id][self.template_version]
        print(max_ncols)
        print(max([len(el) for el in self.df_cols]))
        
        
        
        if max([len(el) for el in self.df_cols]) < max_ncols:
            raise BankStatementColumnsError(self.template_version, self.bank_id, self.mapping)
            log_info('PDF-TABLES',self.account_type, self.bank_id, self.filename,'BankStatementColumnsError')   
        
        elif max([len(el) for el in self.df_cols]) > max_ncols:
            log_info('PDF-TABLES',self.account_type, self.bank_id, self.filename,'BankStatementColumnsWarning')
            self.passed_df_cols_idx = [ind for ind, ex in enumerate(self.df_cols) if len(ex) == 8]
            failed_idx = list(set(list(range(len(self.df_cols))))- set(self.passed_df_cols_idx))
            failed_idx.sort()
            self.not_passed_df_cols_idx = failed_idx
            
        return self.df_cols
    
    
    
    
    
    def black_list_narrow_tables(self, verbose = False):
        '''
        removes the tables that do not contain transaction data
        '''
        
        # each pdf can extract tables
        # (some tables will be without headers)
        # We want to recreate the header (which matches table 1)
        
        
        self.finalized_dfs = []
        self.blacklisted_tables_idxs = []
        
        # all dataframes with statement operations is reformatted
        self.reformatted_dfs = [self.df[k].T.reset_index().T for k in range(1,len(self.df),1)]
        
        
        for idx,r_df in enumerate(self.reformatted_dfs): 
            try:
                # the first dataframe with the right header is the dataframe id = 1
                r_df.columns = self.df[1].columns
                r_df.index = range(r_df.shape[0])
                r_df = r_df.replace(regex=[r'^Unnamed:..$'], value=np.nan)
                
                # store  the cleaned data frame in a placeholder list
                self.finalized_dfs.append(r_df)
                
            except Exception as e:
                if verbose : print(e)
                log_info('DATAFRAME-CLEANING',self.account_type, self.bank_id, self.filename,e) 
                # if it fails, it is because it is a table of another type
                # they will be blacklisted and kept for potential processing
                self.blacklisted_tables_idxs.append(idx)

    
    def combine_dataframes(self):
        '''
        combines dataframes that contain financial data
        '''
        # combine all whitelisted dataframes (=transactions)
        try:
            self.master_df = pd.concat(self.finalized_dfs[:]).reset_index(drop=True)
            log_info('DATAFRAME-COMBINATION',self.account_type, self.bank_id, self.filename, None)
            
            assert (self.master_df.columns == self.df[1].columns), "the columns names do not match the top header"
            assert (self.shape[0] != 0), "the dataframe is empty - no rows"
            
        except Exception as e:
            log_info('DATAFRAME-COMBINATION', self.account_type, self.bank_id, self.filename, e) 

        
    def clean_dataframe(self):
        '''
        simple data cleaning to remove non informative cells
        '''
        # extra cleaning due to columns concatenation
        self.master_df = self.master_df[self.master_df['Remarks']!= 'Remarks']
        self.master_df = self.master_df[self.master_df['Remarks']!= 'Balance as at Last Transaction.']
        self.master_df = self.master_df[self.master_df["Trans. Date"] != 'Trans. Date']
        self.master_df = self.master_df.reset_index(drop=True)
        log_info('CLEAN-DATFRAME', self.account_type, self.bank_id, self.filename, None)
        
        
    def clean_transactions(self):
        '''
        keep only the transactions with dates and add one fictious date for easier processing
        
        '''
        try:
            self.clean_dataframe()
            self.transaction_not_null = self.master_df[~self.master_df["Trans. Date"].isna()].copy()
            max_idx = self.self.transaction_not_null.index.max()
            self.index_list_of_transaction = list(self.transaction_not_null.index)
            self.transaction_not_null.loc[max_idx + 1,'Trans. Date'] = '99-Apr-9999'
            
            log_info('CLEAN-TRANSACTIONS', self.account_type, self.bank_id, self.filename, None) 

            assert (self.transaction_not_null.shape[0] == max_idx + 1),"the additional row  (99-Apr-9999) was not added"
            assert (self.transaction_not_null.loc[max_idx + 1,'Trans. Date'] == '99-Apr-9999'),"the additional row (99-Apr-9999) was not properly added"

        except Exception as e:
            log_info('CLEAN-TRANSACTIONS', self.account_type, self.bank_id, self.filename, e) 
            
    def postprocess(self, verbose = False):
        '''
        reconstruct the financial operations which overflow to the next line in 1 single text
        '''
        
        # all the indexes of the transaction with dates
        self.index_with_dates = self.transaction_not_null.index
        
        self.descr = {}
        for step_date in self.index_with_dates:
            self.descr[str(step_date)] = []
  

        for idx, step in enumerate(self.index_with_dates):
            if idx < len(self.index_with_dates)-1:
                for ind in range(self.index_with_dates[idx], self.index_with_dates[idx+1], 1):
                    if ind < self.index_with_dates[idx+1]: 
                        if str(self.master_df.loc[ind, 'Remarks']) != 'nan':
                            self.descr[str(step)] += [str(self.master_df.loc[ind, 'Remarks'])]
            else:
                for ind in range(self.index_with_dates[idx], self.master_df.shape[0], 1):
                    if str(self.master_df.loc[ind, 'Remarks']) != 'nan':
                        self.descr[str(step)] += [str(self.master_df.loc[ind, 'Remarks'])]
                        
        for key in self.descr.keys():
            self.descr[key] = (''.join(self.descr[key])).replace('\r',' ')
        if verbose:
            return self.descr
    
    def recombined_dataframe(self, verbose= False):
        '''
        reconstruct the final dataset with all original transaction information plus the annotations
        '''
        # Dataframe of the transactions
        annotations = pd.DataFrame.from_dict(self.descr,  orient='index', columns=['Remarks_processed'])
        self.dataset_recombined = pd.concat([self.transaction_not_null.reset_index(drop=True), annotations.reset_index(drop=True)], axis=1)
        if verbose:
            return self.dataset_recombined
        

In [608]:
### Demo on GT bank - template with no header

In [633]:
customer1 =  GT_BankStatement_no_header("./MR SAMSON UGWANYI STATEMENT-part2.pdf")

In [634]:
customer1.describe_accounts_types()

['SAVINGS ACCOUNT']

In [635]:
customer1.log_check_account_type()

In [636]:
customer1.account_type

'SAVINGS ACCOUNT'

In [637]:
customer1.log_check_bank_statement_extension()

In [642]:
customer1.black_list_narrow_tables()

In [643]:
customer1.log_check_table()

8
9


[Index(['Print. Date', '07-Jun-2021'], dtype='object'),
 Index(['Trans. Date', 'Value. Date', 'Reference', 'Debits', 'Credits',
        'Balance', 'Originating Branch', 'Remarks'],
       dtype='object'),
 Index(['03-Jan-2021', '03-Jan-2021.1', ''0MBTR', '5,000.00', 'Unnamed: 0',
        '768,540.74', 'E- CHANNELS',
        'TRANSFER BETWEEN CUSTOMERS MBANKING -\rOffering REF:448632007515232700009377766901\rANY Account Transfer   from UGWUANYI, SAMSON\rONYEBUCHI  to  DOMINION HOUSE'],
       dtype='object'),
 Index(['04-Jan-2021', '04-Jan-2021.1', ''0NIPM', '50.00', 'Unnamed: 0',
        '87,613.87', 'E- CHANNELS',
        'COMMISSION 000013210104215031000322272872\rNIP TRANSFER COMMISSION FOR\r000013210104215031000322272872 Return TO\rFB/COLLINS IGNATIUS AGBO\rReF:000013210104215031000322272872'],
       dtype='object'),
 Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
        'Unnamed: 5', 'Unnamed: 6', 'Generator'],
       dtype='object'),
 Index(['Unnam

In [644]:
customer1.combine_dataframes()

In [645]:
customer1.master_df

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks
0,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks
1,02-Jan-2021,02-Jan-2021,'99993561269999,"10,200.00",NaN,"807,840.74",E- CHANNELS,POS/WEB PURCHASE TRANSACTION -356126- -
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,317852-CAPRICORN DIGITAL LIAL NG
3,02-Jan-2021,02-Jan-2021,'99990126109999,"20,500.00",NaN,"787,340.74",E- CHANNELS,FUNDS TRANSFER -012610- -192340-PAYCOM
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIGERIA LIMITED LALANG
...,...,...,...,...,...,...,...,...
2798,05-Jun-2021,05-Jun-2021,'0USAT,"1,000.00",NaN,"62,975.80",E- CHANNELS,Airtime Purchase MBANKING-\r101CT0000000002645283788-2348124996861
2799,06-Jun-2021,06-Jun-2021,'0NIPM,"5,000.00",NaN,"57,975.80",E- CHANNELS,NIBSS Instant Payment Outward\r000013210606142027000167915088 Labor TO
2800,06-Jun-2021,06-Jun-2021.1,'0NIPM,10.00,NaN,"57,965.80",E- CHANNELS,COMMISSION 000013210606142027000167915088\rNIP TRANSFER COMMISSION FOR\r000013210606142027000167915088 Labor TO\rJAKAWA DANALD DAUDA\rReF:000013210606142027000167915088
2801,06-Jun-2021,06-Jun-2021,'0NIPM,.75,NaN,"57,965.05",E- CHANNELS,VALUE ADDED TAX\r000013210606142027000167915088 VAT ON NIP\rTRANSFER FOR\r000013210606142027000167915088 Labor TO\rJAKAWA DANALD DAUDA\rReF:000013210606142027000167915088


In [646]:
customer1.clean_dataframe()

In [647]:
customer1.clean_transactions()

In [648]:
customer1.postprocess()

In [649]:
customer1.recombined_dataframe()

In [650]:
customer1.dataset_recombined

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks,Remarks_processed
0,02-Jan-2021,02-Jan-2021,'99993561269999,"10,200.00",NaN,"807,840.74",E- CHANNELS,POS/WEB PURCHASE TRANSACTION -356126- -,POS/WEB PURCHASE TRANSACTION -356126- -317852-CAPRICORN DIGITAL LIAL NG
1,02-Jan-2021,02-Jan-2021,'99990126109999,"20,500.00",NaN,"787,340.74",E- CHANNELS,FUNDS TRANSFER -012610- -192340-PAYCOM,FUNDS TRANSFER -012610- -192340-PAYCOMNIGERIA LIMITED LALANG
2,02-Jan-2021,02-Jan-2021,'99990457779999,"7,600.00",NaN,"779,740.74",E- CHANNELS,POS/WEB PURCHASE TRANSACTION -045777- -,POS/WEB PURCHASE TRANSACTION -045777- -294580-EAT 'N' GO LIMITEDLALANG
3,02-Jan-2021,03-Jan-2021,'0USAT,"1,200.00",NaN,"778,540.74",E- CHANNELS,Airtime Purchase MBANKING-,Airtime Purchase MBANKING-101CT0000000002266889748-2348181222235
4,03-Jan-2021,03-Jan-2021.1,'0MBTR,"5,000.00",NaN,"768,540.74",E- CHANNELS,"TRANSFER BETWEEN CUSTOMERS MBANKING -\rOffering REF:448632007515232700009377766901\rANY Account Transfer from UGWUANYI, SAMSON\rONYEBUCHI to DOMINION HOUSE","TRANSFER BETWEEN CUSTOMERS MBANKING - Offering REF:448632007515232700009377766901 ANY Account Transfer from UGWUANYI, SAMSON ONYEBUCHI to DOMINION HOUSE"
5,04-Jan-2021,04-Jan-2021,'99995660409999,"6,850.00",NaN,"761,690.74",E- CHANNELS,POS/WEB PURCHASE TRANSACTION -566040- -\r503401-FORBAYGS INTEGRATED A L NG,POS/WEB PURCHASE TRANSACTION -566040- - 503401-FORBAYGS INTEGRATED A L NG
6,04-Jan-2021,04-Jan-2021,'0NIPM,"10,000.00",NaN,"751,690.74",E- CHANNELS,NIBSS Instant Payment Outward\r000013210104095419000321264916 QUOTA TO\rFB/ST STEPHEN ANGLICAN CHURCH EHA\r/26.875/REF:000013210104095419000321264916,NIBSS Instant Payment Outward 000013210104095419000321264916 QUOTA TO FB/ST STEPHEN ANGLICAN CHURCH EHA /26.875/REF:000013210104095419000321264916
7,04-Jan-2021,04-Jan-2021,'0NIPM,25.00,NaN,"751,665.74",E- CHANNELS,COMMISSION 000013210104095419000321264916\rNIP TRANSFER COMMISSION FOR\r000013210104095419000321264916 QUOTA TO\rFB/ST STEPHEN ANGLICAN CHURCH EHA\rReF:000013210104095419000321264916,COMMISSION 000013210104095419000321264916 NIP TRANSFER COMMISSION FOR 000013210104095419000321264916 QUOTA TO FB/ST STEPHEN ANGLICAN CHURCH EHA ReF:000013210104095419000321264916
8,04-Jan-2021,04-Jan-2021,'0NIPM,1.87,NaN,"751,663.87",E- CHANNELS,VALUE ADDED TAX\r000013210104095419000321264916 VAT ON NIP\rTRANSFER FOR\r000013210104095419000321264916 QUOTA TO\rFB/ST STEPHEN ANGLICAN CHURCH EHA\rReF:000013210104095419000321264916,VALUE ADDED TAX 000013210104095419000321264916 VAT ON NIP TRANSFER FOR 000013210104095419000321264916 QUOTA TO FB/ST STEPHEN ANGLICAN CHURCH EHA ReF:000013210104095419000321264916
9,04-Jan-2021,04-Jan-2021,'99990044939999,"10,000.00",NaN,"741,663.87",E- CHANNELS,FUNDS TRANSFER -004493- -931334-T004493\r2TGT7463 LA LANG,FUNDS TRANSFER -004493- -931334-T004493 2TGT7463 LA LANG


In [651]:
customer1.dataset_recombined = customer1.dataset_recombined[['Trans. Date','Reference','Value. Date','Debits','Credits','Balance','Remarks_processed']]

## 2. Features engineering (-> classifier)

In [732]:
class Dataset_Bank():
    def __init__(self, Data, a_list_of_keywords):
        # save data inside the object
        self._data = Data
        self.features_engineering(a_list_of_keywords)
        self.filter_text()
        self.dest()  
        self.clean_data()
        self.save2csv(out="./first_data.csv")
        self.subset_dataframe()
        #print(self.df.columns())
        self.impute_class()
        
    def features_engineering(self, a_list_of_keywords):

        for key_word in tqdm(a_list_of_keywords):
            self.check_keyword(key_word)
    
    def check_keyword(self, keyword):
        # check whether the criteria is valid
        self._data["IS_A_"+ keyword] =  self._data.loc[:,"Remarks_processed"].str.contains(keyword, case=False).astype('int')
    
    def filter_text(self):
        # filter description
        self._data["filtered_description"] = self._data.loc[:,"Remarks_processed"].str.findall('[^\d\W]+').str.join(sep =' ')
    
    def dest(self):
        # filtered benificiary
        self._data["filtered_dest"] = self._data["filtered_description"].str.upper()
        self._data["filtered_dest"] = self._data["filtered_dest"].str.split("TO").str[-1]
        
    def clean_data(self):
        # fix data problems
        # the "date format" problem inside timestamp cell
        # problem found : 02-Apr-2021.1
        csv_to_annotate = self._data
        for c in csv_to_annotate.columns[:2]:
            csv_to_annotate[c] = csv_to_annotate[c].apply(lambda x : str(x).split(".")[0])
            
        # problem found : \r inside the dataframe
        csv_to_annotate[c] = csv_to_annotate[c].astype('str').str.replace("\r"," ")
        
    def save2csv(self, out):
        self.df = csv_to_annotate.iloc[:-1,:]
        self.df.to_csv(out, index= None, sep = ";",encoding='utf-8-sig')
        print(self.df.columns)
        
        
    def subset_dataframe(self):
        '''
        creates subset dataframes with key information (loan, salary, cheque,commission,tax)
        '''
        self.salary = self.df[self.df["Remarks_processed"].str.contains("SALARY", case=False).values]
        self.loan = self.df[self.df["Remarks_processed"].str.contains("LOAN", case=False).values]
        self.tax = self.df[self.df["Remarks_processed"].str.contains("TAX", case=False).values]
        self.commission = self.df[self.df["Remarks_processed"].str.contains("COMM", case=False).values]
        self.cheque = self.df[self.df["Remarks_processed"].str.contains("CHEQUE",case=False).values]
        
        
    def impute_class(self):
        '''
        classify transactions based on remarks
        '''
        
        self.subset_dataframe()
        comm_index = self.commission.index
        tax_index = self.tax.index
        salary_index = self.salary.index
        loan_index = self.loan.index
        
        impute_list = [[comm_index, "COMM"],
                       [tax_index,"TAX"],
                       [salary_index,"SALARY"],
                      [loan_index,"LOAN"]]

        self.df_class = self.df.copy()
        for element in impute_list:
            self.df_class.loc[element[0],"Class"] = element[1]

        return self.df_class           


In [779]:
class Metrics():
    def __init__(self,imputed_Dataset):
        self.Ds = imputed_Dataset
        self.data = imputed_Dataset.df_class
        self.create_loan_summary()
        
    def create_loan_summary(self):
        # convert entries to date format
        self.data.loc[:]["Trans. Date"] = pd.to_datetime(self.Ds.loan["Trans. Date"], errors='coerce')
        
        # create a summary limiting ourselves to some columns
        loan_summary = self.Ds.loan.loc[:,["Trans. Date", "Credits", "Debits"]].copy()
        
        # use the colum ( just transformed to date format as index)
        loan_summary.set_index('Trans. Date', inplace=True)
        
        
        # make sure thzat the index is @ the right format (date)
        loan_summary.index = pd.to_datetime(loan_summary.index)
        
        
        # remove the missing value and replace them by 0
        for cat in ["Credits", "Debits"]:
            loan_summary[cat].fillna("0", inplace=True)

        
        # remove the thousands separator
        for cat in ["Credits", "Debits"]:
            loan_summary[cat]= loan_summary[cat].str.replace(",","").astype('float')
            
            
        # "effective" number of loans
        self.df_loan_summary = loan_summary.groupby(pd.Grouper(level='Trans. Date',freq='M')).sum()
        
        credits = self.df_loan_summary.copy()
        debits = self.df_loan_summary.copy()

        credits[credits['Credits']>0]=-1
        debits[debits['Debits']>0]=1
        
        df_credits = credits[['Credits']]
        df_debits = debits[['Debits']]
        
        self.total_n_loans_summary = pd.DataFrame(pd.concat([df_debits,df_credits], axis = 1).sum(axis=1))
        self.total_n_loans_summary.columns = ['#n loans']
        self.total_n_loans_summary['#n loans'] = self.total_n_loans_summary['#n loans'].apply(lambda x: np.abs(x))
        
        self.df_loan_summary["effective"] = self.df_loan_summary["Debits"]-self.df_loan_summary["Credits"]
        
        return self.df_loan_summary, self.total_n_loans_summary
        

In [780]:
list_of_keywords = ["TRANSFER", "TRANSFER COMMISSION","TRANSFER BETWEEN CUSTOMERS","CONTRIBUTION",
                   "TAX","SALARY", "VAT", "WEB PURCHASE", "POS/WEB",
                   "AIRTIME", "PURCHASE", "INSTANT PAYMENT",
                   "CASH","WITHDRAWAL", "ATM", "SMS ALERT CHARGE",
                   "FAILED", "FUNDS", "FAILED","FUNDS","LOAN","INTEREST",
                    "GTWORLD", "LOANDISBURSEMENT"]


GT_Dataset = Dataset_Bank(customer1.dataset_recombined,list_of_keywords)


100%|██████████| 24/24 [00:00<00:00, 330.50it/s]

Index(['Trans. Date', 'Reference', 'Value. Date', 'Debits', 'Credits',
       'Balance', 'Remarks_processed', 'IS_A_TRANSFER',
       'IS_A_TRANSFER COMMISSION', 'IS_A_CONTRIBUTION', 'IS_A_TAX',
       'IS_A_SALARY', 'IS_A_VAT', 'IS_A_WEB PURCHASE', 'IS_A_POS/WEB',
       'IS_A_AIRTIME', 'IS_A_PURCHASE', 'IS_A_INSTANT PAYMENT', 'IS_A_CASH',
       'IS_A_WITHDRAWAL', 'IS_A_ATM', 'IS_A_SMS ALERT CHARGE', 'IS_A_FAILED',
       'IS_A_FUNDS', 'IS_A_LOAN', 'IS_A_INTEREST', 'IS_A_GTWORLD',
       'filtered_description', 'filtered_dest',
       'IS_A_TRANSFER BETWEEN CUSTOMERS', 'IS_A_LOANDISBURSEMENT'],
      dtype='object')


In [781]:
GT_Dataset.loan

,Trans. Date,Reference,Value. Date,Debits,Credits,Balance,Remarks_processed,IS_A_TRANSFER,IS_A_TRANSFER COMMISSION,IS_A_CONTRIBUTION,IS_A_TAX,IS_A_SALARY,IS_A_VAT,IS_A_WEB PURCHASE,IS_A_POS/WEB,IS_A_AIRTIME,IS_A_PURCHASE,IS_A_INSTANT PAYMENT,IS_A_CASH,IS_A_WITHDRAWAL,IS_A_ATM,IS_A_SMS ALERT CHARGE,IS_A_FAILED,IS_A_FUNDS,IS_A_LOAN,IS_A_INTEREST,IS_A_GTWORLD,filtered_description,filtered_dest,IS_A_TRANSFER BETWEEN CUSTOMERS,IS_A_LOANDISBURSEMENT
41,13-Jan-2021,'0GWTR,13-Jan-2021,nan,"200,000.00","200,045.63","TRANSFER BETWEEN CUSTOMERS ViaGAPSLITE Being additional loan facility for onemonthREF:208851059000002000002101131354from RODA CREST LIMITED to UGWUANYI,SAMSON ONYEBUCHI",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,TRANSFER BETWEEN CUSTOMERS ViaGAPSLITE Being additional loan facility for onemonthREF from RODA CREST LIMITED to UGWUANYI SAMSON ONYEBUCHI,UGWUANYI SAMSON ONYEBUCHI,1,0
136,05-Feb-2021,'0NIPT,05-Feb-2021,nan,"221,267.13","223,834.42",TRANSFER BETWEEN CUSTOMERS000016210205150519000070865626|CIB:LOANDISBURSMENT|637481343190170043||LIBERTALEASING LIMITEDREF:000016210205150519000070865626,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,TRANSFER BETWEEN CUSTOMERS CIB LOANDISBURSMENT LIBERTALEASING LIMITEDREF,MERS CIB LOANDISBURSMENT LIBERTALEASING LIMITEDREF,1,0
556,12-Apr-2021,'0NIPT,12-Apr-2021,nan,"180,000.00","195,711.99",TRANSFER BETWEEN CUSTOMERS 000014210412114838230832826550|PP:Asset Loan / GTB / UGWUANYI SAMSON ONYEBUCHI/UPS STAFF COOP.38838|PP_NIP_1011266885_0||UPS STAFF COOP.38838 REF:000014210412114838230832826550,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,TRANSFER BETWEEN CUSTOMERS PP Asset Loan GTB UGWUANYI SAMSON ONYEBUCHI UPS STAFF COOP PP_NIP_ _ UPS STAFF COOP REF,MERS PP ASSET LOAN GTB UGWUANYI SAMSON ONYEBUCHI UPS STAFF COOP PP_NIP_ _ UPS STAFF COOP REF,1,0
719,30-Apr-2021,'0,30-Apr-2021.1,nan,"300,000.00","329,249.95",TRANSFER BETWEEN CUSTOMERS BEING ONE OFF LOAN REQUEST FRM RODA CREST LTD TO SAMSON UGWUANYI,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,TRANSFER BETWEEN CUSTOMERS BEING ONE OFF LOAN REQUEST FRM RODA CREST LTD TO SAMSON UGWUANYI,SAMSON UGWUANYI,1,0


In [782]:
loans_summary, n_loans_summary = Metrics(GT_Dataset).create_loan_summary()

In [783]:
# to do check against a certain regexp
# unit tests

In [784]:
loans_summary

,Credits,Debits,effective
Trans. Date,,,
2021-01-31,200000.00,0.0,-200000.00
2021-02-28,221267.13,0.0,-221267.13
2021-03-31,0.00,0.0,0.00
2021-04-30,480000.00,0.0,-480000.00


In [785]:
n_loans_summary

,#n loans
Trans. Date,
2021-01-31,1.0
2021-02-28,1.0
2021-03-31,0.0
2021-04-30,1.0


In [ ]:
# loans - raw summary

In [ ]:
# "effective" number of loans

In [199]:
df_loan_summary = loan_summary.groupby(pd.Grouper(level='Trans. Date',freq='M')).sum()

In [200]:
credits = df_loan_summary.copy()
debits = df_loan_summary.copy()

credits[credits['Credits']>0]=-1
debits[debits['Debits']>0]=1

In [201]:
df_credits = credits[['Credits']]
df_debits = debits[['Debits']]

In [206]:
total_n_loans_summary = pd.DataFrame(pd.concat([df_debits,df_credits], axis = 1).sum(axis=1))
total_n_loans_summary.columns = ['#n loans']

In [207]:
# "effective" loans

In [208]:
df_loan_summary["effective"] = df_loan_summary["Debits"]-df_loan_summary["Credits"]

In [209]:
df_loan_summary

,Credits,Debits,effective
Trans. Date,,,
2021-04-30,0.00,19261.72,19261.72
2021-05-31,0.00,19261.72,19261.72
2021-06-30,1321.71,1321.71,0.00


In [210]:
total_n_loans_summary

,#n loans
Trans. Date,
2021-04-30,1.0
2021-05-31,1.0
2021-06-30,0.0


In [126]:
salary = df[df["Remarks"].str.contains("SALARY").values]

### 3. Analytics

In [127]:
# salary

In [128]:
salary.loc[:]["Trans. Date"] = pd.to_datetime(salary["Trans. Date"], errors='coerce')

In [129]:
salary.loc[:,["Trans. Date", "Credits"]]

,Trans. Date,Credits
441,28-Apr-2021,"52,605.90"
442,28-Apr-2021,"10,000.00"
451,28-May-2021,"52,605.90"
452,28-May-2021,"10,000.00"
461,28-Jun-2021,"10,000.00"
462,28-Jun-2021,"52,605.90"


In [130]:
salary

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Class,Remarks,Remarks_processed,IS_A_TRANSFER,IS_A_TRANSFER COMMISSION,IS_A_CONTRIBUTION,IS_A_TAX,IS_A_SALARY,IS_A_VAT,IS_A_WEB PURCHASE,IS_A_POS/WEB,IS_A_AIRTIME,IS_A_PURCHASE,IS_A_INSTANT PAYMENT,IS_A_CASH,IS_A_WITHDRAWAL,IS_A_ATM,IS_A_SMS ALERT CHARGE,IS_A_FAILED,IS_A_FUNDS,IS_A_LOAN,IS_A_INTEREST,IS_A_GTWORLD,filtered_description,filtered_dest
441,28-Apr-2021,28-Apr-2021,66126644GAPS,NaN,"52,605.90","36,632.03",E- CHANNELS,LOAN,MTHLY SALARY via GAPS 112121953 GTBank,MTHLY SALARY via GAPS 112121953 GTBankSalary for April 2021 from STRESERT SERVICESLIMITED to ADELANA PAUL OLANREWAJUREF:0001701329112121953000000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,MTHLY SALARY via GAPS GTBankSalary for April from STRESERT SERVICESLIMITED to ADELANA PAUL OLANREWAJUREF,ADELANA PAUL OLANREWAJUREF
442,28-Apr-2021,28-Apr-2021,66128260GAPS,NaN,"10,000.00","46,632.03",E- CHANNELS,LOAN,MTHLY SALARY via GAPS 112123898 GTBank Bonus for April 2021 from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329112123898000000,MTHLY SALARY via GAPS 112123898 GTBank Bonus for April 2021 from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329112123898000000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,MTHLY SALARY via GAPS GTBank Bonus for April from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF,ADELANA PAUL OLANREWAJU REF
451,28-May-2021,28-May-2021,69548935GAPK,NaN,"52,605.90","34,876.21",E- CHANNELS,LOAN,MTHLY SALARY via GAPS 113722179 May 2021 Salary for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329113722179000000,MTHLY SALARY via GAPS 113722179 May 2021 Salary for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329113722179000000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,MTHLY SALARY via GAPS May Salary for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF,ADELANA PAUL OLANREWAJU REF
452,28-May-2021,28-May-2021,69549381GAPS,NaN,"10,000.00","44,876.21",E- CHANNELS,LOAN,MTHLY SALARY via GAPS 113722651 May 2021 Bonus for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329113722651000000,MTHLY SALARY via GAPS 113722651 May 2021 Bonus for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329113722651000000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,MTHLY SALARY via GAPS May Bonus for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF,ADELANA PAUL OLANREWAJU REF
461,28-Jun-2021,28-Jun-2021,72865794GAPS,NaN,"10,000.00","-5,106.93",E- CHANNELS,LOAN,MTHLY SALARY via GAPS 115442885 GTBank Bonus for June 2021 from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329115442885000000,MTHLY SALARY via GAPS 115442885 GTBank Bonus for June 2021 from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329115442885000000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,MTHLY SALARY via GAPS GTBank Bonus for June from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF,ADELANA PAUL OLANREWAJU REF
462,28-Jun-2021,28-Jun-2021,72867714GAPS,NaN,"52,605.90","47,498.97",E- CHANNELS,LOAN,MTHLY SALARY via GAPS 115445208 GTBank June 2021 Staff Salary from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329115445208000000,MTHLY SALARY via GAPS 115445208 GTBank June 2021 Staff Salary from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329115445208000000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,Fals

In [131]:
salary.set_index('Trans. Date', inplace=True)

In [132]:
salary.index = pd.to_datetime(salary.index)

In [133]:
number_of_payments = len(set(list(salary.index)))
paid_month = [el.month for el in set(list(salary.index))]
paid_days = [el.day for el in set(list(salary.index))]

paid_day = np.mean(paid_days)  

variance_in_paid_days = np.std(paid_days) 

In [134]:
salary_numeric = salary.loc[:,["Debits","Credits"]]

In [135]:
salary_numeric["Credits"]= salary_numeric["Credits"].str.replace(",","").astype('float')

In [136]:
df_salary = salary_numeric.groupby(pd.Grouper(level='Trans. Date',freq='M')).sum()

In [140]:
df_salary

,Credits
Trans. Date,
2021-04-30,62605.9
2021-05-31,62605.9
2021-06-30,62605.9


In [137]:
df_salary.mean()

Credits    62605.9
dtype: float64

In [155]:
def salary_stability(df_salary):
    return df_salary['Credits'].std()

In [158]:
def loan_to_salary_ratio(df_salary, df_loan):
    
    df_loan['effective']
    df_salary['Credits']
    loan_to_salary = pd.merge(df_loan['effective'],df_salary['Credits'], left_index=True,right_index=True )
    loan_to_salary.columns = ['effective_loan', 'salary']
    loan_to_salary['ratio'] = loan_to_salary['effective_loan']/loan_to_salary['salary']
    return loan_to_salary

In [159]:
salary_stability(df_salary)

0.0

In [160]:
loan_to_salary_ratio(df_salary, df_loan_summary)

,effective_loan,salary,ratio
Trans. Date,,,
2021-04-30,19261.72,62605.9,0.307666
2021-05-31,19261.72,62605.9,0.307666
2021-06-30,0.00,62605.9,0.000000


In [153]:
def salary_bracket(salary_dollars):
    if salary_dollars > 50000 and salary_dollars < 200000:
        bracket = "LOW_EARNER"
    elif salary_dollars > 210000 and salary_dollars < 500000:
        bracket = "LOW_MED_EARNER"
    elif salary_dollars > 500000 and salary_dollars < 1000000:  
        bracket = "MED_EARNER"
    elif salary_dollars > 1000000 and salary_dollars < 3000000: 
        bracket = "HIGH_MED_EARNER"
    elif salary_dollars > 3000000 and salary_dollars < 5000000:
        bracket = "HIGH_EARNER"
    elif salary_dollars > 5000000:
        bracket = "TOP_EARNER"
    return bracket

In [154]:
DTI = {
"LOW_EARNER": 0.33,
"LOW_MED_EARNER":0.35,
"MED_EARNER":0.40,
"HIGH_MED_EARNER":0.45,
"HIGH_EARNER": 0.5,
"TOP_EARNER": 0.55
}

## 3. Rules

In [ ]:
# high risk:
- Low Income
- Multiple Loans
- Total Monthly Loan repayment is higher than Net salary
- Instant transfer of salary to any other account

In [ ]:
# move the money on the other account the day or the next day" not many transactions in the account
# probaly something to hide
# 12465 type of expense - hidden things for a loan
# direct debit /loan

In [ ]:
# Ayo

- Unstable Net Income ( what is the criteria for instable : variation of how many % ?) - > 10 % & above
- leave allowances/bonus (quaterly) -tricky

- Unstable Salary Date ( what is the criteria for instable : variation of how many days ?) More than 1 week

- Constant delay repayment (where to find this info?) - can not do (finsoft)

- No access to direct debit on salary Account (where to find this info?)
- no direct debit (standard charter, first bank, stanbic)

- already taken I.O.U from Organization (not clear)
- company name (mentionned for salary)

- High Number of returned cheques 
- 1 max for any period

- Works in a blue chip company - can not do myself (finsoft)
- (structured company)

In [ ]:
# Assan

- Salary Account is not a checking account  (to debug)

In [211]:
pd.read_csv('./dataset_statement_bank_type.csv')

,bank,folder,filename,bank_account,template_type
0,gt_bank,1_statement,214773647637581511468199616204-part1.pdf,CURRENT,NO_HEADER
1,gt_bank,1_statement,214773647637581511468199616204-part2.pdf,SAVINGS,NO_HEADER
2,gt_bank,1_statement,214773647637581511468199616204-part3.pdf,CURRENT,NO_HEADER
3,gt_bank,1_statement,214773647637581511468199616204-part4.pdf,MASTERCARD/VISA DEBIT CARD,NO_HEADER
4,gt_bank,1_statement,250231923637586605609611256130-part1,CURRENT,NO_HEADER
5,gt_bank,1_statement,250231923637586605609611256130-part2,SAVINGS,NO_HEADER
6,gt_bank,1_statement,Mr festus oshokhue statement-part1,CURRENT,NO_HEADER
7,gt_bank,1_statement,Mr festus oshokhue statement-part2,SAVINGS,NO_HEADER
8,gt_bank,1_statement,MR SAMSON UGWANYI STATEMENT-part1,CURRENT,NO_HEADER
9,gt_bank,1_statement,MR SAMSON UGWANYI STATEMENT-part2,SAVINGS,NO_HEADER


## 4. Classifier Model

## 5 score extraction

### a. Credit score (final score)

In [ ]:
l_credit_scores = glob2.glob('./credit_score/*')

In [ ]:
credit_report_godwin = l_credit_scores[0]

In [ ]:
TOP = 253
LEFT = 23
WIDTH = 280
HEIGHT = 28
PDF_WIDTH = 612
PDF_HEIGHT = 792



df = tabula.read_pdf(credit_report_godwin, 
                     multiple_tables=True,
                     area=[100*TOP/PDF_HEIGHT,
                           100*LEFT/PDF_WIDTH,
                           100*(TOP+ HEIGHT)/PDF_HEIGHT,
                           100*(LEFT+WIDTH)/PDF_WIDTH],
                     relative_area=True,
                    pages='1')

In [ ]:
df[0]

### b. Score summary breakdown

In [ ]:
# repayment history
# total amount owed
# types of credits in use


TOP = 196
LEFT = 244
WIDTH = 112
HEIGHT = 54
PDF_WIDTH = 612
PDF_HEIGHT = 792



df = tabula.read_pdf(credit_report_godwin, 
                     multiple_tables=True,
                     area=[100*TOP/PDF_HEIGHT,
                           100*LEFT/PDF_WIDTH,
                           100*(TOP+ HEIGHT)/PDF_HEIGHT,
                           100*(LEFT+WIDTH)/PDF_WIDTH],
                     relative_area=True,
                    pages='1')

score_summary_breakdown_1 = df[0].T.reset_index(drop=True).T

In [ ]:
# length of credit history
# number of credits



TOP = 180
LEFT = 356
WIDTH = 116
HEIGHT = 53
PDF_WIDTH = 612
PDF_HEIGHT = 792



df = tabula.read_pdf(credit_report_godwin, 
                     multiple_tables=True,
                     area=[100*TOP/PDF_HEIGHT,
                           100*LEFT/PDF_WIDTH,
                           100*(TOP+ HEIGHT)/PDF_HEIGHT,
                           100*(LEFT+WIDTH)/PDF_WIDTH],
                     relative_area=True,
                    pages='1')

score_summary_breakdown_2 = df[0].T.reset_index(drop=True).T

In [ ]:
score_summary_breakdown = pd.concat([score_summary_breakdown_1,score_summary_breakdown_2], axis=0).T

In [ ]:
new_header = [el.strip() for el in score_summary_breakdown.iloc[0]]
score_summary = score_summary_breakdown[1:]
score_summary.columns = new_header

In [ ]:
score_summary

In [ ]:
score_total_amount_owned = score_summary['Total Amount Owed:'].str.split("/").str[0].astype('float')
score_total_type_of_credits_in_use = score_summary['Types of Credit(s) in Use:'].str.split("/").str[0].astype('float')
score_total_length_of_credit_history = score_summary['Length of Credit History:'].str.split("/").str[0].astype('float')
score_total_number_of_credits = score_summary['Number of Credits:'].str.split("/").str[0].astype('float')

In [ ]:
score_total_amount_owned_max = score_summary['Total Amount Owed:'].str.split("/").str[1].astype('float')
score_total_type_of_credits_in_use_max = score_summary['Types of Credit(s) in Use:'].str.split("/").str[1].astype('float')
score_total_length_of_credit_history_max = score_summary['Length of Credit History:'].str.split("/").str[1].astype('float')
score_total_number_of_credits_max = score_summary['Number of Credits:'].str.split("/").str[1].astype('float')

In [ ]:
pd.DataFrame([score_total_amount_owned/score_total_amount_owned_max,
score_total_type_of_credits_in_use/score_total_type_of_credits_in_use_max,
score_total_length_of_credit_history/score_total_length_of_credit_history_max,
score_total_number_of_credits/score_total_number_of_credits_max]).T

___________

In [ ]:
# each pdf can extract tables
# (those tables will be without headers)

In [ ]:
# this first table has the correct header
df[1].columns

In [ ]:
# this second table DOES NOT have the correct header
df[2].columns

In [ ]:
# We want to recreate the header (which matches table 1)

In [ ]:
reformatted_dfs = [df[k].T.reset_index().T for k in range(1,len(df),1)]
finalized_dfs = []
for idx,r_df in enumerate(reformatted_dfs): 
    try:
        r_df.columns = df[1].columns
        r_df.index = range(r_df.shape[0])
        r_df = r_df.replace(regex=[r'^Unnamed:..$'], value=np.nan)
        finalized_dfs.append(r_df)
    except:
        # if it fails, it is because it is a table of another type
        print(idx)

In [ ]:
# Visualize the first table of the finalized dataframes
finalized_dfs[0]

In [ ]:
# reset index in order to prevent problmems of indexation
master_df = pd.concat(finalized_dfs[:]).reset_index(drop=True)

In [ ]:
# extra cleaning

master_df = master_df[master_df['Remarks']!= 'Remarks']
master_df = master_df[master_df['Remarks']!= 'Balance as at Last Transaction.']
master_df = master_df[master_df["Trans. Date"] != 'Trans. Date']

In [ ]:
master_df = master_df.reset_index(drop=True)

In [ ]:
transaction_not_null = master_df[~master_df["Trans. Date"].isna()]
index_list_of_transaction = list(transaction_not_null.index)

In [ ]:
# all the transaction whithout a null starting date
transaction_not_null

In [ ]:
# check that the column names is OK
transaction_not_null.columns

In [ ]:
# create a dictionary with row indexes to keep
descr = {}

In [ ]:
#transaction_not_null.loc[transaction_not_null.shape[0]]= [np.nan for i in range(8)]
# Create an arbitrary date (year 4021)
transaction_not_null.loc[master_df.index.max() +1,'Trans. Date'] = '01-Apr-4021'
#transaction_not_null = transaction_not_null[transaction_not_null["Trans. Date"] != 'Trans. Date']

In [ ]:
# visualize the dataframe and its modification
transaction_not_null 

In [ ]:
# all the indexes of the transaction with dates
index_with_dates = transaction_not_null.index

In [ ]:
descr ={}
for step in index_with_dates:
    descr[str(step)] = []
descr

In [ ]:
step = 0    

for idx, step in enumerate(index_with_dates):
    if idx < len(index_with_dates)-1:
        for ind in range(index_with_dates[idx], index_with_dates[idx+1], 1):
            if ind < index_with_dates[idx+1]: 
                if str(master_df.loc[ind, 'Remarks']) != 'nan':
                    descr[str(step)] += [str(master_df.loc[ind, 'Remarks'])]
    else:
        for ind in range(index_with_dates[idx], master_df.shape[0], 1):
            if str(master_df.loc[ind, 'Remarks']) != 'nan':
                descr[str(step)] += [str(master_df.loc[ind, 'Remarks'])]

In [ ]:
for key in descr.keys():
    descr[key] = (''.join(descr[key])).replace('\r',' ')

In [ ]:
descr

In [ ]:
# Dataframe of the transactions
pd.DataFrame.from_dict(descr,  orient='index', columns=['remarks_one_line'])

In [ ]:
import json

In [ ]:
my_j =  "{\"Transactions\": [{\"transactionType\": \"SALES\",\"transactionAmount\": 10.05}]}"

In [ ]:
json.loads(my_j)

In [ ]:
my_j['msg'].keys()

In [ ]:
# trial on 1 dataframe

In [ ]:
first_transaction = df[1]

first_transaction_not_null = first_transaction[~first_transaction["Trans. Date"].isna()]
index_list_of_transaction = list(first_transaction_not_null.index)
first_transaction_not_null.columns

In [ ]:
first_transaction

In [ ]:
descr = {}
first_transaction.shape

In [ ]:
first_transaction.loc[first_transaction.shape[0]]= [np.nan for i in range(8)]
first_transaction.loc[first_transaction.shape[0],'Trans. Date'] = '01-Apr-4021'

In [ ]:
first_transaction

In [ ]:
index_with_dates = list(first_transaction[~first_transaction['Trans. Date'].isnull()].index)

In [ ]:
descr ={}
for step in index_with_dates:
    descr[step] = []
descr

In [ ]:
index_with_dates

In [ ]:

step = 0    

for idx, step in enumerate(index_with_dates):
    print(idx)
    if idx < len(index_with_dates)-1:
        for ind in range(index_with_dates[idx], index_with_dates[idx+1], 1):
            if ind < index_with_dates[idx+1]: 
                if str(first_transaction.loc[ind, 'Remarks']) != 'nan':
                    descr[step] += [str(first_transaction.loc[ind, 'Remarks'])]
    else:
        for ind in range(index_with_dates[idx], first_transaction.shape[0], 1):
            if str(first_transaction.loc[ind, 'Remarks']) != 'nan':
                descr[step] += [str(first_transaction.loc[ind, 'Remarks'])]

In [ ]:
for key in descr.keys():
    descr[key] = (''.join(descr[key]))

In [ ]:
descr

# 2. Data conversion

In [ ]:
import time
import typing

In [ ]:
def time_my_func(func):
    
    def inner_func(a_string):
        start = time.time()
        a =  func(a_string)
        end = time.time()
        print(f"it took : {(end-start)*1e6} microseconds")
        return a
    return inner_func
        

In [ ]:
@time_my_func
def process(my_string:str) -> int:
    return(len(my_string))

In [ ]:
process("this sheez is crazy")

In [ ]:
def parameterize(c):
    def smart_div(func):
        def inner_f(a,b):
            return(func(c,a))
        return inner_f
    return smart_div

In [ ]:
@parameterize(500)
def div(a,b):
    return a/b

In [ ]:
div(3,4)

In [ ]:

div(30,4)

# 3. Conversion evaluation